# Цель занятия
На этом занятии мы рассмотрим применение алгоритма KNN для решения задач регрессии и классификации с подбором параметра K.

## Регрессия

Для примера регрессии с K-ближайших соседей (KNN) с подбором количества ближайших соседей (K) в библиотеке scikit-learn мы можем использовать набор данных по ценам на недвижимость в Бостоне. Мы будем использовать модуль GridSearchCV для выбора оптимального значения параметра K.

Boston dataset - это набор данных, содержащий информацию о недвижимости в городе Бостон, штат Массачусетс, США. Датасет состоит из 506 строк и 14 столбцов.

Каждая строка представляет собой информацию об одном районе Бостона, а каждый столбец - это различные характеристики этого района. Вот описание каждого столбца:

- CRIM: Уровень преступности на душу населения в районе
- ZN: Доля земельных участков для жилой застройки площадью более 25 000 кв. футов
- INDUS: Доля площадей, занятых не жилой застройкой в районе
- CHAS: Фиктивная переменная (1, если участок граничит с рекой, иначе 0)
- NOX: Концентрация оксидов азота (частей на 10 миллионов)
- RM: Среднее количество комнат в доме
- AGE: Доля занимаемых владельцами жилья единиц, построенных до 1940 года
- DIS: Взвешенное расстояние до пяти бостонских центров занятости
- RAD: Индекс доступности радиальных магистралей
- TAX: Ставка налога на имущество за 10 000 долларов США
- PTRATIO: Соотношение учеников и учителей в районе
- B: 1000 * (доля афроамериканцев в районе)
- LSTAT: Доля населения с более низким социальным статусом
- TARGET: Средняя стоимость занимаемых владельцами жилья единиц в 1000 долларов США

In [ ]:
# Необходимые библиотеки
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection as model_selection
from sklearn.metrics import mean_squared_error

In [ ]:
# Загрузка датасета: Для начала загрузим датасет в Pandas DataFrame.
# Добавим имена столбцов (column_names)
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
                'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'TARGET']
df = pd.read_csv('datasets/lecture_02_code_labs_03_knn.csv', 
                   header=None,
                   delimiter=r"\s+", 
                   names=column_names)

In [ ]:
# Давайте проверим первые 5 строк датасета, чтобы убедиться, что загрузка прошла успешно.
print(df.head())

In [ ]:
# Очистка данных: Проверим наличие пропущенных значений в датасете.
# Если найдены пропущенные значения, то их можно удалить или заполнить средним, медианой или модой столбца.
# В нашем случае пропущенных данных нет
print(df.isnull().sum())

Визуализация данных: Используем библиотеку seaborn (sns) для визуализации данных. Например, построим scatter plot между двумя столбцами.

In [ ]:
sns.scatterplot(x='RM', y='TARGET', data=df)

Предобработка данных: Для того, чтобы подготовить данные для обучения модели машинного обучения, их необходимо обработать. Например, мы можем нормализовать данные используя StandardScaler из библиотеки scikit-learn.

In [ ]:
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

В библиотеке scikit-learn есть класс KNeighborsRegressor для решения задач регрессии с помощью алгоритма k-NN. Для подбора значения k можно использовать класс GridSearchCV.

In [ ]:
# Разделение на признаки и метки
y = df['TARGET']
X = df.drop(['TARGET'], axis=1)

# Разделение на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# Определение значений параметров для подбора
param_grid = {'n_neighbors': range(1, 20)}

# Создание модели k-NN
knn = KNeighborsRegressor(metric='minkowski', p=2)

# Создание объекта GridSearchCV для подбора параметра k
grid = model_selection.GridSearchCV(knn, param_grid, cv=5)

# Обучение модели на тренировочных данных
grid.fit(X_train, y_train)

# Определение лучшего значения k
best_k = grid.best_params_['n_neighbors']

# Прогнозирование целевых меток на тестовых данных
y_pred = grid.predict(X_test)

# Вычисление MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE для k = {best_k}: {mse}")

## Классификация

Классификация с использованием метода ближайших соседей (k-Nearest Neighbors, KNN) является одним из простых и популярных методов машинного обучения. 

Пример использования KNN с подбором параметра k с помощью GridSearchCV в библиотеке scikit-learn представлен ниже. 
В этом примере мы используем набор данных Iris для классификации трех видов ирисов на основе их длины и ширины лепестков и чашелистников. Мы разбиваем данные на обучающую и тестовую выборки, создаем объект KNeighborsClassifier, определяем диапазон значений параметра k, который нужно проверить, и инициализируем объект GridSearchCV для перебора параметров с использованием кросс-валидации. Затем мы обучаем модель на обучающей выборке с использованием GridSearchCV для поиска оптимального значения k и создаем новую модель KNN с оптимальным значением K. В качестве метрики используется accuracy.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Загружаем набор данных Iris
iris = load_iris()
X = iris.data
y = iris.target

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем модель KNN
knn = KNeighborsClassifier()

# Определяем диапазон значений параметра k, которые нужно проверить
k_range = list(range(1, 31))

# Определяем сетку параметров, которые нужно проверить
param_grid = dict(n_neighbors=k_range)

# Инициализируем объект GridSearchCV для перебора параметров с использованием кросс-валидации
grid = model_selection.GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

# Обучаем модель на обучающей выборке с использованием GridSearchCV для поиска оптимального значения k
grid.fit(X_train, y_train)

# Получаем наилучшее значение k
best_k = grid.best_params_['n_neighbors']

# Создаем новую модель KNN с оптимальным значением k
knn_best = KNeighborsClassifier(n_neighbors=best_k)

# Обучаем модель на обучающей выборке с оптимальным значением k
knn_best.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred = knn_best.predict(X_test)

# Оцениваем точность предсказаний на тестовой выборке
accuracy = accuracy_score(y_test, y_pred)

print("Оптимальное значение k:", best_k)
print("Точность предсказаний на тестовой выборке:", accuracy)